<a href="https://colab.research.google.com/github/Vferreira91/Pluviometro/blob/main/Heatmap_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gmaps

In [ ]:
import gmaps
import gmaps.datasets
import pandas as pd
import time
from google.colab import output
from google.colab import drive
from IPython.display import display, clear_output
output.enable_custom_widget_manager()
drive.mount('/content/drive')
gmaps.configure(api_key='[SUA API GMAPS AQUI]')

In [3]:
df = pd.read_csv('/local/do/arquivo.csv')
coord_alt = pd.read_csv('/local/do/arquivo.csv')

In [4]:
#Utilizar as coordenadas alternativas no lugar das originais (não executar
# caso queira utilizar as coordenadas originais)
df = df.drop(['latitude','longitude'],axis=1)
df = df.merge(coord_alt[['id_estacao', 'latitude', 'longitude']], on='id_estacao', how='left')
df = df.drop(['id_estacao'],axis=1)

In [5]:
ordem_colunas = ['tick','latitude','longitude','acumulado']
df = df[ordem_colunas]

In [6]:
locations = df[['latitude', 'longitude']]
acumulado = df['acumulado']

In [ ]:
fig = gmaps.figure(center=(-22.910, -43.450), zoom_level=11.85,
                   layout = {'width': '1800px','height':'1000px', 'margin':'0 auto 0 auto'})

# Inicializador da camada de heatmap
heatmap_layer = gmaps.heatmap_layer(locations, weights=acumulado, max_intensity=200)
# Camada de heatmap à figura do mapa
fig.add_layer(heatmap_layer)
# Mapa inicial
display(fig)

# Atualizar a camada do heatmap com base no dia a cada intervalo de tempo setado no time.sleep
def update_heatmap():
    for tick in sorted(df['tick'].unique()):
        filtered_df = df[df['tick'] == tick]
        heatmap_layer.locations = filtered_df[['latitude', 'longitude']]
        heatmap_layer.weights = filtered_df['acumulado']
        heatmap_layer.max_intensity = 200
        heatmap_layer.point_radius = 130
        heatmap_layer.dissipating = True
        heatmap_layer.gradient = [(0,0,255,0.01),(0,255,255),(173,255,47),
                                (154,205,50),(255,255,0),(255,165,0),(255,69,0),
                                 (255,0,0),(128,0,128)]
        #Diminuir frequencia de atualização em valores maiores
        if heatmap_layer.weights > [90]:
            time.sleep(1.2)  # Itera por 1 'tick' a cada frame (60Hz = 0.016)
        else:
            time.sleep(0.25)
        clear_output(wait=True)  # Limpar tick anterior

update_heatmap()
display(fig)